In [13]:
import pandas as pd
import numpy as np
import os
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

In [11]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.environ["OPENAI_API_KEY"]

In [7]:
df = pd.read_csv(r"C:\AI-ML\campus_x\INTERVIEW_PREP\Onboarding\data\main.csv")
df.head()

,id,completion_date,partner_name,deal_type,review_type,escalation_type,EDD_reviewer,EDD_measures,timestamp
0,1,01-01-2023,Partner A,imt,fresh_onboarding,other_red_flags,neelima_routhu,document_review,01-01-2023 10:00
1,10,10-01-2023,Partner J,payments,periodic_review,other_red_flags,rohan_vazapully,aml_review,10-01-2023 19:00
2,11,11-01-2023,Partner K,issuance,fresh_onboarding,adverse_media,rahil_fw,audit_review,11-01-2023 10:00
3,12,12-01-2023,Partner L,vendor,periodic_review,high_risk,moustapha,feedback_review,12-01-2023 11:00
4,13,13-01-2023,Partner M,imt,fresh_onboarding,pep_association,laura_castillo,document_review,13-01-2023 12:00


In [18]:
chat = ChatOpenAI(openai_api_key = API_KEY, model_name= "gpt-3.5", temperature= 0.0)
agent = create_pandas_dataframe_agent(chat, df, allow_dangerous_code=True)

In [25]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model= "gpt-3.5-turbo"),
    df,
    verbose=True,
    agent_type= AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

In [27]:
agent.run("Give me the names of the EDD_reviewer")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['EDD_reviewer'].tolist()"}`


['neelima_routhu', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'laura_castillo', 'hityshi', 'neelima_routhu', 'francis_xavier', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'francis_xavier', 'laura_castillo', 'hityshi', 'neelima_routhu', 'francis_xavier', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'laura_castillo', 'hityshi', 'neelima_routhu', 'rohan_vazapully', 'francis_xavier', 'neelima_routhu', 'neelima_routhu', 'francis_xavier', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'laura_castillo', 'hityshi', 'neelima_routhu', 'francis_xavier', 'rohan_vazapully', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'laura_castillo', 'hityshi', 'neelima_routhu', 'francis_xavier', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'laura_castillo', 'neelima_routhu', 'francis_xavier', 'rohan_vazapully', 'rahil_fw', 'moustapha', 'laura_castillo', 'hityshi', 'neelima_routhu', 'francis_xavier', 

'The names of the EDD_reviewers are:\n- neelima_routhu\n- rohan_vazapully\n- rahil_fw\n- moustapha\n- laura_castillo\n- hityshi\n- francis_xavier'

In [28]:
agent.run("I want to know how many escalations francis_xavier did?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['EDD_reviewer'] == 'francis_xavier']['escalation_type'].count()"}`


17Francis Xavier did 17 escalations.

> Finished chain.


'Francis Xavier did 17 escalations.'

In [29]:
agent.run("who is the best performing reviewer in terms of escalation and edd measures combined?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['total_review_count'] = df['escalation_type'] + df['EDD_measures']\nbest_reviewer = df.groupby('EDD_reviewer')['total_review_count'].sum().idxmax()\nbest_reviewer"}`


francis_xavierThe best performing reviewer in terms of escalation and EDD measures combined is Francis Xavier.

> Finished chain.


'The best performing reviewer in terms of escalation and EDD measures combined is Francis Xavier.'

In [33]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
import pandas as pd
import mysql.connector

In [45]:
llm = ChatOpenAI(temperature= 0, model = "gpt-3.5-turbo")
file = "florida-deathrow.sqlite"

In [37]:
# Replace these with your actual MySQL server details
MYSQL_HOST = "localhost"
MYSQL_USER = "root"
MYSQL_PASSWORD = "root"
MYSQL_DATABASE = "TERRAPAY"

def fetch_data():
    try:
        conn = mysql.connector.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DATABASE
        )
        query = "SELECT * FROM onboarding"
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    except mysql.connector.Error as err:
        print(f"Error fetching data: {err}")
        return pd.DataFrame()

In [ ]:
data = fetch_data()
data.head(2)

In [46]:
connection_uri = f"mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{3306}/{MYSQL_DATABASE}"

In [49]:
db = SQLDatabase.from_uri("connection_uri")
toolkit=SQLDatabaseToolkit(db=db, llm=llm)

In [51]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
    
)

In [52]:
agent_executor.run("describe the table")



> Entering new SQL Agent Executor chain...
I should use the sql_db_schema tool to get the schema and sample rows for the table
Action: sql_db_schema
Action Input: table_nameError: table_names {'table_name'} not found in databaseI should first check the list of tables in the database to ensure I am using the correct table name
Action: sql_db_list_tables
Action Input: entries, onboarding, uploadsI should now use the sql_db_schema tool with the correct table name
Action: sql_db_schema
Action Input: entries
CREATE TABLE entries (
	id VARCHAR(255) NOT NULL, 
	completion_date DATE, 
	partner_name VARCHAR(255), 
	deal_type VARCHAR(255), 
	review_type VARCHAR(255), 
	escalation_type VARCHAR(255), 
	`EDD_reviewer` VARCHAR(255), 
	`EDD_measures` VARCHAR(255), 
	timestamp TIMESTAMP NULL, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from entries table:
id	completion_date	partner_name	deal_type	review_type	escalation_type	EDD_reviewer	EDD_measure

'The "entries" table has columns id, completion_date, partner_name, deal_type, review_type, escalation_type, EDD_reviewer, EDD_measures, and timestamp.'

In [53]:
agent_executor.run("find the best performing reviewer from the onboarding table for the month of may")



> Entering new SQL Agent Executor chain...
I need to first check the tables available in the database and then query the onboarding table to find the best performing reviewer for the month of May.
Action: sql_db_list_tables
Action Input: entries, onboarding, uploadsI need to check the schema of the onboarding table to see what fields are available for me to use in my query.
Action: sql_db_schema
Action Input: onboarding
CREATE TABLE onboarding (
	id VARCHAR(255) NOT NULL, 
	completion_date DATE, 
	partner_name VARCHAR(255), 
	deal_type VARCHAR(255), 
	review_type VARCHAR(255), 
	escalation_type VARCHAR(255), 
	`EDD_reviewer` VARCHAR(255), 
	`EDD_measures` VARCHAR(255), 
	timestamp TIMESTAMP NULL, 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from onboarding table:
id	completion_date	partner_name	deal_type	review_type	escalation_type	EDD_reviewer	EDD_measures	timestamp
1	2023-01-01	Partner A	imt	fresh_onboarding	other_red_flags	neelima

'neelima_routhu'